# Method Characteriation #
## Goal ##
The goal of this test set is to perform proof of concept testing on a small number of proteins with a wide range of sizes and available homologs, orthologs, and paralogs. By doing so it should be possible to test the best parameterization for this tool as well as identifying the strengths and weaknesses of the tool using various measurments as end points.
## Warning ##
Before attempting to use this notebook make sure that your .env file has been properly setup to reflect the correct locations of command line tools and the location of files and directories needed for execution.
### Initial Import###
This first cell performs the necessary imports required to begin this notebook.

In [ ]:
from dotenv import find_dotenv, load_dotenv
try:
    dotenv_path = find_dotenv(raise_error_if_not_found=True)
except IOError:
    dotenv_path = find_dotenv(raise_error_if_not_found=True, usecwd=True)
load_dotenv(dotenv_path)
import os
import sys
sys.path.append(os.path.join(os.environ.get('PROJECT_PATH'), 'src'))
input_dir = os.environ.get('INPUT_PATH')

## Data Set Construction ##
The first task required to test the data set is to download the required data and construct any necessary input files for all down stream analyses.
In this case that means:
* Downloading PDB files for the proteins in our small test set.
* Extracting a query sequence from each PDB file.
* Searching for paralogs, homologs, and orthologs in a custom BLAST database built by filtering the Uniref90 database.
* Filtering the hits from the BLAST search to meet minimum and maximum length requirements, as well as minimum and maximum identity requirements.
* Building alignments using CLUSTALW in both the fasta and msf formats since some of the tools which will be used for comparison need different formats.
* Filtering the alignment for maximum identity similarity between seqeunces.
* Re-aligning the filtered sequences using CLUSTALW.
This is all handeled by the DataSetGenerator class found in the src/SupportingClasses folder

In [ ]:
from time import time
from Supporting_Classes.DataSetGenerator import DataSetGenerator
protein_list_dir = os.path.join(input_dir, 'ProteinLists')
if not os.path.isdir(protein_list_dir):
    os.makedirs(protein_list_dir)
small_list_fn = os.path.join(protein_list_dir, 'SmallDataSet.txt')
if not os.path.isfile(small_list_fn):
    proteins_of_interest = ['2ysdA', '1c17A', '3tnuA', '7hvpA', '135lA', '206lA', '2b59A', '2werA', '1bolA', '3q05A',
                            '1axbA', '2rh1A', '1hckA', '3b6vA', '2z0eA', '1jwlA', '1a26A', '1c0kA', '1h1vA', '4lliA',
                            '4ycuA', '2iopA', '2zxeA']
    with open(small_list_fn, 'wb') as small_list_handle:
        for p_id in proteins_of_interest:
            small_list_handle.write('{}\n'.format(p_id))
generator = DataSetGenerator(input_dir)
start = time()
dataset_df = generator.build_pdb_alignment_dataset(protein_list_fn=os.path.basename(small_list_fn), num_threads=10,
                                                   database='customuniref90.fasta', max_target_seqs=2500, remote=False,
                                                   verbose=False)
dataset_df['Length'] = dataset_df['Protein_ID'].apply(lambda x: int(data_dict[x]['Length']),
                                                      args=(generator.protein_data,))
dataset_df['Total_Size'] = dataset_df.apply(lambda row: row['Length'] * row['Filtered_Alignment'], axis=1)
dataset_df.sort(columns=['Total_Size', 'Length', 'Filtered_Alignment'], ascending=[1, 1, 1], axis='columns', inplace=True)
end = time()
print('It took {} min to generate the data set.'.format((end - start) / 60.0))

Create a location to store the output of this parameter tuning.

In [ ]:
output_dir = os.environ.get('OUTPUT_PATH')
characterization_out_dir = os.path.join(output_dir, 'Characterization')
if not os.path.isdir(characterization_out_dir):
    os.makedirs(characterization_out_dir)

## Method Characterization##
This section performs the Evoluationary Trace method for covariation ('pair's of residues) with different parameters to help determine which provide the optimal behavior (in this case measured by ability to predict structural contacts: AUROC, Precision, and clustering: Biased SCW Z-Score, Unbiased SCW Z-Score).
### Distance Metric Parameters###
* identity', False - Uses the identity metric to compute the distance between the sequences in the provided alignment.
* 'blosum62', True - Uses the similarity metric (as defined by the off diagonal values from the 'blosum62' distance matrix) to compute the distance between the sequences in the provided alignment.
* 'blosum62', False - Uses the 'blosum62' scoring matrix to compute the edit distance between sequences in the provided alignment.
### Tree Construction Parameters###
* 'et' - A phylogenetic tree related to the UPGMA tree, but where the distance update does not use the average of columns from the current step, but rather the average distance between all contributing terminal nodes. This has been used in previous methods by our group.
* 'upgma' - A phylogenetic tree constructed using the standard UPGMA algorithm.
* 'agglomerative' (affinity='euclidean', linkage='ward') - A tree constructed based on agglomerative/hierarchical clustering over the distance matrix using the specified affinity and linkage.
### Scoring Parameters###
* 'identity' - A binary scoring of invariance at each level and node in the tree, which yields an integer score for each pair of positions (lower score means the pair position was fixed higher in the tree and therefore more important, higher score means the pair of positions became fixed lower down in the score and is therefore less important).
* 'plain_entropy' - The joint entropy between a pair of positions. This provides a real valued (floating point score) for each pair of positions, which can be interpreted the same way as the 'identity' scoring metric but with greater resolution/ability to separate positions.
* 'mutual_information' - The mutual informaiton score for each pair of positions within a node of the phylogenetic tree. This is built up over levels of the tree using the trace methodology, in this case the higher the score the better.
* 'normalized_mutual_information' - The normalized mutual informaiton score for each pair of positions within a node of the phylogenetic tree. This is built up over levels of the tree using the trace methodology, in this case the higher the score the better.
* 'average_product_corrected_mutual_information' - The average product corrected mutual informaiton (MIp) score for each pair of positions within a node of the phylogenetic tree. This is built up over levels of the tree using the trace methodology, in this case the higher the score the better (some low scores my be negative).
* 'filtered_average_product_corrected_mutual_information' - The average product corrected mutual informaiton (MIp) score for each pair of positions within a node of the phylogenetic tree. This is built up over levels of the tree using the trace methodology, in this case the higher the score the better (some low scores my be negative). This score is additionally filtered at the node level so that positions with a mutual information <= 0.0001 are set to 0 (this was done in a previously released paper by our lab on this topic).

In [ ]:
from EvolutionaryTrace import EvolutionaryTrace
from SupportingClasses.ContactScorer import ContactScorer
characterization_scores = {'Protein': [], 'Distance': [], 'Tree': [], 'Scoring': [], 'Separation': [], 'AUROC': [],
                           'Max_Unbiased_SCW_Z-Score': [], 'AUC_Unbiased_SCW_Z-Score': [],
                           'Max_Biased_SCW_Z-Score': [], 'AUC_Biased_SCW_Z-Score': []}
for p_id in dataset_df['Protein_ID']:
    contact_scorer = None
    biased_w2_ave = None
    unbiased_w2_ave = None
    print('Characterizing protein: {}'.format(p_id))
    protein_dir = os.path.join(characterization_out_dir, p_id)
    if not os.path.isdir(protein_dir):
        os.mkdir(protein_dir)
    for dist_model, et_dist in [('identity', False), ('blosum62', True), ('blosum62', False)]:
        print('Distance model: {}, ET dist: {}'.format(dist_model, et_dist))
        for tree_building, tree_options in [('et', {}), ('upgma', {}),
                                            ('agglomerative', {'affinity': 'euclidean', 'linkage': 'ward'})]:
            dist_tree_dir = os.path.join(
                protein_dir, '{}{}{}{}'.format(dist_model, '_ET' if et_dist else ''), tree_building,
                ('_'.join(['{}_{}'.format(k, v) for k,v in tree_options.items()]) if tree_options else ''))
            if not os.path.isdir(dist_tree_dir):
                os.mkdir(dist_tree_dir)
            for scoring_metric in ['identity', 'plain_entropy', 'mutual_information', 'normalized_mutual_information',
                                   'average_product_corrected_mutual_information',
                                   'filtered_average_product_corrected_mutual_information']:
                scoring_dir = os.path.join(dist_tree_dir, scoring_metric)
                if not os.path.isdir(scoring_dir):
                    os.mkdir(scoring_dir)
                curr_et = EvolutionaryTrace(query_id=p_id, polymer_type='Protein',
                                            aln_fn=generator.protein_data[p_id]['Final_FA_Aln'], et_distance=et_dist,
                                            distance_model=dist_model, tree_building_method=tree_building,
                                            tree_building_options=tree_options, ranks=None, position_type='pair',
                                            scoring_metric=scoring_metric, gap_correction=None, out_dir=protein_dir,
                                            output_files={'original_aln', 'non_gap_aln', 'tree', 'scores'},
                                            processors=10, low_memory=True)
                curr_et.import_and_process_aln()
                curr_et.out_dir = dist_tree_dir
                curr_et.compute_distance_matrix_tree_and_assignments()
                curr_et.out_dir = scoring_dir
                curr_et.perform_trace()
                if contact_scorer is None:
                    pdb_structure = PDBReference(pdb_file=generator.protein_data[p_id]['PDB'])
                    pdb_structure.import_pdb()
                    contact_scorer = ContactScorer(seq_alignment=curr_et.non_gapped_aln,
                                                   pdb_reference=pdb_structure, cutoff=8.0)
                    contact_scorer.best_chain = generator.protein_data[p_id]['Chain']
                    contact_scorer.fit()
                z_score_fn = os.path.join(scoring_dir, '{}_ZScores.tsv')
                z_score_biased, b_w2_ave, b_scw_z_auc = self.score_clustering_of_contact_predictions(
                    scores, bias=True, file_path=z_score_fn.format('Biased'), w2_ave_sub=biased_w2_ave,
                    processes=processes)
                max_biased_z_score = np.max(pd.to_numeric(z_score_biased['Z-Score'], errors='coerce'))
                if (biased_w2_ave is None) and (b_w2_ave is not None):
                    biased_w2_ave = b_w2_ave
                z_score_unbiased, u_w2_ave, u_scw_z_auc = self.score_clustering_of_contact_predictions(
                    scores, bias=False, file_path=z_score_fn.format('Unbiased'), w2_ave_sub=unbiased_w2_ave,
                    processes=processes)
                max_unbiased_z_score = np.max(pd.to_numeric(z_score_unbiased['Z-Score'], errors='coerce'))
                if (unbiased_w2_ave is None) and (u_w2_ave is not None):
                    unbiased_w2_ave = u_w2_ave
                for separation in ['Any', 'Neighbors', 'Short', 'Medium', 'Long']:
                    _, _, auroc = contact_scorer.score_auc(1.0 - curr_et.coverage, category=separation)
                    for k in range(1, 11):
                        if k == 1:
                            precision_label = 'Precision (L)'
                        else:
                            precision_label = 'Precision (L/{})'.format(k)
                        if precision_label not in characterization_scores:
                            stats[precision_label] = []
                        precision = contact_scorer.score_precision(predictions=scores, k=k, category=separation)
                        characterization_scores['Protein'].append(p_id)
                        characterization_scores['Distance'].append('{}{}'.format(dist_model, '_ET' if et_dist else '')))
                        characterization_scores['Tree'].append('{}{}'.format(tree_building,
                                                                             ('_'.join(['{}_{}'.format(k, v)
                                                                                        for k,v in tree_options.items()])
                                                                              if tree_options else '')))
                        characterization_scores['Scoring'].append(scoring_metric)
                        characterization_scores['Separation'].append(separation)
                        characterization_scores['AUROC'].append(auroc)
                        characterization_scores[precision_label].append(precision)
                        characterization_scores['Max_Biased_SCW_Z-Score'].append(max_biased_z_score)
                        characterization_scores['AUC_Biased_SCW_Z-Score'].append(b_scw_z_auc)
                        characterization_scores['Max_Unbiased_SCW_Z-Score'].append(max_unbiased_z_score)
                        characterization_scores['AUC_Unbiased_SCW_Z-Score'].append(u_scw_z_auc)